In [1]:
# Valuation of European Call Options using Fast Fourier Transform applied to Lewis Formula (2001)
# Models: BSM and H93

#Import necessary packages
import math
import numpy as np
from numpy.fft import fft

#Characteristic Functions

def BSM_characteristic_function(v, T, r, d, sigma):
    '''  Valuation of European call option in BSM model via
    Lewis (2001) and Carr-Madan (1999)
    --> Fourier-based approach: charcteristic function. '''
    cf_value = np.exp((- 0.5 * sigma ** 2 * 1j * v
                - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value

def H93_char_func(u, T, r, d, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach: characteristic function.
    Parameter definitions see function BCC_call_value.'''
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v)
            ** 2 - sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) \
          / (kappa_v - rho * sigma_v * u * 1j - c2)
    H1 = ((r-d) * u * 1j * T + (c1 / sigma_v ** 2)
          * ((kappa_v - rho * sigma_v * u * 1j + c2) * T
                - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v ** 2
          * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    char_func_value = np.exp(H1 + H2 * v0)*np.exp(- 1j*u*(r-d)*T)
    return char_func_value

#Valuation by FFT for BSM model

def BSM_call_value_FFT(S0, K, T, r, d, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001)
    --> Fourier-based approach (integral).
    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    d: float
        dividen yield
    sigma: float
        volatility factor in diffusion term
    Returns
    =======
    call_value: float
        European call option present value
    '''
    k = np.log(S0/K)+(r-d)*T
    g = 1                         # factor to increase accuracy  
    N = g * 4096
    eps = (g * 150.) ** -1
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k         # range extreme value for k
    u = np.arange(1, N + 1, 1)
    
    # Grid values
    v1 = -N/2*eta+eta/2
    vN = N/2*eta-eta/2
    k1 = -b
    vu = v1 + eta * (u - 1)                
    v = -vu-0.5*1j
    
    # Numerical FFT Routine
    delt = np.zeros(N, dtype=np.float)    
    delt[0] = 1                            # Kronecker's delta
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3  # Simpson's weights
    FFTFunc = (np.exp(-1j * v1 * k) * eta* np.exp(-1j * k1 * (u-1)*eta) * BSM_characteristic_function(v, T, r, d, sigma)/(vu**2+0.25)) * SimpsonW
    payoff = (fft(FFTFunc)).real           # Fast Fourier Transform
    CallValueM = S0*np.exp(-d*T)-np.sqrt(S0*K)*np.exp(-(r+d)*T/2)*payoff / (2*np.pi)
    pos = int((k+b)/eps)
    CallValue = CallValueM[pos]

    return CallValue 

                                        
def H93_call_value_FFT(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via
    Carr-Madan (1999) Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    d: float
        dividend yield
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance

    Returns
    =======
    call_value: float
        European call option present value
    '''
    
    k = np.log(S0/K)+(r-d)*T
    g = 1                         # factor to increase accuracy  
    N = g * 4096
    eps = (g * 150.) ** -1
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k         # range extreme value for k
    u = np.arange(1, N + 1, 1)
    
    # Grid values
    v1 = -N/2*eta+eta/2
    vN = N/2*eta-eta/2
    k1 = -b
    vu = v1 + eta * (u - 1)                
    v = -vu-0.5*1j
    
    # Numerical FFT Routine
    delt = np.zeros(N, dtype=np.float)    
    delt[0] = 1                            # Kronecker's delta
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3  # Simpson's weights
    FFTFunc = (np.exp(-1j * k1 * (u-1)*eta) * H93_char_func(v, T, r, d, kappa_v, theta_v, sigma_v, rho, v0)/(vu**2+0.25)) * SimpsonW
    payoff = (np.exp(-1j * v1 * k) * eta* fft(FFTFunc)).real     # Fast Fourier Transform
    CallValueM = S0*np.exp(-d*T)-np.sqrt(S0*K)*np.exp(-(r+d)*T/2)*payoff / (2*np.pi)
    pos = int((k+b) / eps)
    CallValue = CallValueM[pos]
    
    return CallValue 
 